## Filtro de dados no databricks

https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.filter.html

Filtrar dados é uma etapa fundamental na análise de dados, permitindo selecionar apenas as informações relevantes de um DataFrame. No Databricks, existem diversas formas de aplicar filtros utilizando métodos do PySpark, comandos SQL e funções específicas para manipulação de strings e valores. Nesta seção, vamos explorar as principais técnicas para filtrar dados de maneira eficiente e flexível.


In [0]:
path = "/Volumes/workspace/default/tutorial/anac"
df = spark.read.json(f"{path}/V_OCORRENCIA_AMPLA.json")
display(df)

In [0]:
# Vamos filtrar apenas alguns dados relevantes
df = df.select("UF", "Classificacao_da_Ocorrência")

### Filtros de texto (Filter)

O método `.filter()` do PySpark DataFrame permite selecionar linhas que atendem a uma condição específica, retornando um novo DataFrame apenas com os dados filtrados. Ele aceita como parâmetro uma expressão booleana, que pode ser passada como uma string SQL ou como uma expressão construída com funções do PySpark. Os principais parâmetros são:

- **condition**: Expressão booleana que define o critério de filtragem. Pode ser uma string (ex: `"coluna = 'valor'"`) ou uma expressão (`col("coluna") == "valor"`).

- **subset** *(opcional, em alguns contextos)*: Lista de colunas a serem consideradas na filtragem, geralmente usada em funções auxiliares.

Exemplo de uso:

```py
df.filter("idade > 18")
df.filter(col("nome").startswith("A"))
```

In [0]:
# Vizualizando os dados sem salvar em um DF novo ou sobrescrever o antigo 
display(df.filter(df.UF == "MG"))

In [0]:
# Salvando em um novo DF
df_MG = df.filter(df.UF == "MG")
display(df_MG)


### Operadores lógicos

* **&** = "E" lógico,  tem que cumprir todos os criterios para retornar o resultado 1° filtro e o 2° filtro o que tiver resultado igual nos 2 criterios ele vai trazer 

* **|** = "OU" lógico, quando obedecer um criterio ou o outro ele vai trazer 

* **!=** = NÃO lógico, quando o critério for o oposto do inserido.

Nomes:

* **&** = e comercial
* **|** = "barra vertical" ou "pipe". 

In [0]:
# Passar cada Criterio emtre parenteses com o operador lógico no meio para separar os criterios 
display(df.filter((df.UF == "MG") & (df.Classificacao_da_Ocorrência == "Incidente")))

In [0]:
display(df.filter((df.Danos_a_Aeronave == "leve") | (df.Classificacao_da_Ocorrência == "Incidente")))

In [0]:
display(df.filter((df.Danos_a_Aeronave == "Leve") & (df.Classificacao_da_Ocorrência == "Incidente")))

In [0]:
# Salvando em um Df novo
dfnovo = df.filter((df.Danos_a_Aeronave != "Leve") & (df.Classificacao_da_Ocorrência == "Incidente"))
display(dfnovo)

### Metodo where

O método `.where()` do PySpark DataFrame é utilizado para filtrar linhas com base em condições específicas, funcionando de maneira equivalente ao método `.filter()`. Ele permite selecionar apenas os dados que atendem a determinados critérios, facilitando a análise e manipulação de grandes volumes de dados.

**Principais parâmetros do `.where`:**

- **condition**: Expressão booleana que define o critério de filtragem. Pode ser uma string SQL (ex: `"coluna = 'valor'"`) ou uma expressão construída com funções do PySpark (ex: `col("coluna") > 10`).

- **subset** *(opcional, em alguns contextos)*: Lista de colunas a serem consideradas na filtragem, geralmente usada em funções auxiliares.

Exemplo de uso:

```py
df.where("idade >= 21")
df.where(col("nome").contains("Silva"))
```

#### Diferença entre `.filter()` e `.where()`

No PySpark, os métodos `.filter()` e `.where()` são funcionalmente equivalentes: ambos servem para filtrar linhas de um DataFrame com base em uma condição booleana. Não há diferença de desempenho ou sintaxe entre eles, sendo a escolha apenas uma questão de preferência ou legibilidade do código.

Exemplo equivalente:

```py
df.filter(df["idade"] > 18)
df.where(df["idade"] > 18)
```

In [0]:
# Visualizando os dados nao estou salvando em lugar nenhum 
display(df.where(df.UF == 'SP'))

In [0]:
display(df.where((df.Danos_a_Aeronave == "Leve") & (df.Classificacao_da_Ocorrência == "Incidente")))

In [0]:
# Salvando em um DF
DfFiltrado = df.where((df.Danos_a_Aeronave == "Leve") & (df.Classificacao_da_Ocorrência == "Incidente"))
display(DfFiltrado)

### Método Comando SQL

Documentação https://spark.apache.org/docs/latest/api/sql/

O comando `.sql` no Databricks permite executar consultas SQL diretamente em notebooks, facilitando a análise e manipulação de dados usando a linguagem SQL padrão. Esse método é útil para quem já está familiarizado com SQL e deseja realizar operações como seleção, filtragem, agregação e transformação de dados de forma rápida e intuitiva.

**Principais parâmetros do `.sql`:**

- **query**: (obrigatório) String contendo a consulta SQL a ser executada. Exemplo: `"SELECT * FROM tabela WHERE coluna = 'valor'"`.

- **args**: (opcional) Dicionário de parâmetros para interpolação na query, permitindo consultas dinâmicas.

- **database**: (opcional) Nome do banco de dados a ser utilizado como contexto padrão para a consulta.

- **options**: (opcional) Dicionário com opções adicionais de execução, como configurações de timeout ou limites de resultados.

Exemplo de uso:
```py
spark.sql("SELECT nome, idade FROM pessoas WHERE idade > 18")
```

In [0]:
#Criar uma tabela temporaria
df.createOrReplaceTempView("temp_teste")
mg = spark.sql('''
SELECT 
  `Classificacao_da_Ocorrência` as Classificacao,
  Descricao_do_Tipo             as Tipo,
  Fase_da_Operacao              as Fase,
  Municipio,
  UF                            as Estado
FROM temp_teste
WHERE UF = "MG" 
and Fase_da_Operacao = "Decolagem"
''')

display(mg)

In [0]:
# Exemplo sem as 3 aspas simples ou Duplas
mg = spark.sql('SELECT * FROM temp_teste WHERE UF = "MG"')
display(mg)

In [0]:
%sql
show databases

In [0]:
%sql
show tables

In [0]:
%sql
DROP TABLE IF EXISTS temp_teste;


### Functions do Spark

As funções do Spark são essenciais para manipulação, transformação e análise de dados em DataFrames e consultas SQL. Elas abrangem desde operações matemáticas, manipulação de strings, datas, arrays, até funções de agregação e criação de funções personalizadas (UDFs). O uso eficiente dessas funções permite realizar análises complexas de forma simples e performática no Databricks e em ambientes Spark em geral.

Documentação functions

https://spark.apache.org/docs/latest/sql-ref-functions.html

Documentação functions string

https://spark.apache.org/docs/latest/sql-ref-functions-builtin.html#string-functions


O PySpark fornece uma ampla gama de funções dentro do módulo `pyspark.sql.functions`.
Alguns Exemplos

| Função | Descrição | Exemplo |
|--------|-----------|---------|
| **col** | Retorna uma coluna do DataFrame com base no nome fornecido. | `col("nome_da_coluna")` |
| **concat** | Concatena várias colunas em uma única coluna. | `concat(col("coluna1"), lit(" "), col("coluna2"))` |
| **substring** | Retorna uma parte de uma coluna de texto com base em índices. | `substring(col("texto"), 1, 3)` |
| **length** | Retorna o comprimento de uma coluna de texto. | `length(col("texto"))` |
| **lower** | Converte o texto de uma coluna para minúsculas. | `lower(col("texto"))` |
| **upper** | Converte o texto de uma coluna para maiúsculas. | `upper(col("texto"))` |
| **trim** | Remove espaços em branco do início e do final de uma coluna de texto. | `trim(col("texto"))` |
| **replace** | Substitui um padrão por outro em uma coluna de texto. | `replace(col("texto"), "old", "new")` |
| **regexp_replace** | Substitui um padrão de expressão regular por outro em uma coluna de texto. | `regexp_replace(col("texto"), "pattern", "replacement")` |
| **split** | Divide uma coluna de texto em um array com base em um delimitador. | `split(col("texto"), " ")` |
| **substring_index** | Retorna as primeiras n ocorrências de um delimitador em uma coluna de texto. | `substring_index(col("texto"), " ", 2)` |
| **concat_ws** | Concatena colunas usando um delimitador especificado. | `concat_ws(" ", col("coluna1"), col("coluna2"))` |
| **when** | Realiza uma operação condicional em uma coluna com base em uma condição. | `when(col("idade") >= 18, "adulto").otherwise("menor")` |
| **coalesce** | Retorna a primeira coluna não nula de um conjunto de colunas. | `coalesce(col("coluna1"), col("coluna2"), col("coluna3"))` |

Essas são apenas algumas das muitas funções disponíveis no módulo `pyspark.sql.functions`

In [0]:
from pyspark.sql.functions import *


In [0]:
# Retorna uma coluna do DataFrame passando nome como referencia
display(df.filter(col('UF') == 'SP'))

### Filtrando pelo indice da coluna

Em algumas situações, pode ser necessário filtrar dados de um DataFrame utilizando o índice (posição) das colunas, especialmente quando não se conhece previamente o nome das colunas ou quando se deseja aplicar filtros de forma dinâmica. No PySpark, é possível acessar colunas por índice utilizando a lista de nomes de colunas do DataFrame, permitindo construir filtros flexíveis e adaptáveis a diferentes estruturas de dados.

In [0]:
colunas = df.columns
display(colunas)

In [0]:
# Localizando Indice em Dados com varias colunas 
Coluna = "Operador"
Indice = df.columns.index(Coluna)
display(Indice)

In [0]:
# Gera um dataframe com a lista de colunas e seus indices
ListaIndices = []

# Loop 
for Coluna in df.columns:
    Indice = df.columns.index(Coluna)
    ListaIndices.append((Coluna, Indice))

# Salvar lista em DF
df_indices = spark.createDataFrame(ListaIndices, ["Coluna", "Indice"])

display(df_indices)


In [0]:
# Filtrando pelo indice 
df_filtrado = df.filter(col(df.columns[43]) == "SCF-PP") 
display(df_filtrado)


In [0]:
# Inserindo coluna e filtro por variaveis
Coluna1 = 'Fase_da_Operacao'
Filtro1 = 'Decolagem'
Coluna2 = 'UF'
Filtro2 = 'MG'

display(df.filter( (df[Coluna1] == Filtro1) & (df[Coluna2] == Filtro2)))

### Corrigindo erro de maiúsculas e minúsculas

Ao trabalhar com dados textuais em DataFrames, é comum encontrar inconsistências relacionadas ao uso de letras maiúsculas e minúsculas, o que pode impactar buscas, comparações e filtragens. Para garantir resultados precisos e evitar erros, é importante padronizar o formato dos textos, convertendo-os para caixa alta (maiúsculas) ou caixa baixa (minúsculas) antes de realizar operações de análise. O PySpark oferece funções específicas para facilitar essa padronização de forma eficiente.

* **lower**: Converte o texto de uma coluna para minúsculas.
* **upper**: Converte o texto de uma coluna para maiúsculas.

In [0]:
display(df.filter(df.Fase_da_Operacao == 'Decolagem'))

In [0]:
display(df.filter(df.Fase_da_Operacao == 'decolagem'))

In [0]:
from pyspark.sql.functions import upper, lower

In [0]:
# Correção para bases grandes
# 1° verificar comportamento 
# 2° criar um padrão para as colunas

display(df.select("Fase_da_Operacao").distinct())


In [0]:
# Upper  = Transforma dados da coluna em maiúsculas
display(df.select(upper("Fase_da_Operacao")).distinct())

In [0]:
# lower  = Transforma dados da coluna em minúsculas 
display(df.select(lower("Fase_da_Operacao")).distinct())

In [0]:
# Filtro sem erro eu resolvi fazer minúsculas
display(df.filter(lower(df.Fase_da_Operacao) == 'decolagem'))

In [0]:
# Notem que no dataframe Permanece como Maiusculo, usei somente no filtro para nao haver erro. O DF permanece original depois vamos aprender a alterar direto na base 
novo_df = df.filter(lower(df.Fase_da_Operacao) == 'decolagem')
display(novo_df)

### Resolvendo erros de Digitação com Espaços Excessivos

Ao lidar com dados textuais, é comum encontrar registros com espaços em branco indesejados no início, no final ou até mesmo entre as palavras. Esses espaços podem causar problemas em buscas, comparações e análises, levando a resultados incorretos ou inconsistentes. Nesta seção, vamos abordar técnicas para identificar e remover espaços excessivos em colunas de texto utilizando funções do PySpark, garantindo maior precisão e padronização nos dados.

* **trim**: Remove espaços em branco do início e do final de uma coluna de texto.
* **rtrim**: Remove espaços em branco da direita 
* **ltrim**: Remove espaços em branco da esquerda

In [0]:
Dados = [("Belo Horizonte ", "MG", 2512070),
        (" Belo Horizonte ", "MG", 12106920),
        ("Rio de Janeiro", "RJ", 6718903),
        (" Belo Horizonte", "MG", 20001151),
        ("Brasília", "DF", 3015268)]

Colunas = ["Cidade", "UF", "População"]

new_df = spark.createDataFrame(Dados, Colunas)
display(new_df)

In [0]:
display(new_df.filter(new_df.Cidade == 'Belo Horizonte'))

In [0]:
from pyspark.sql.functions import rtrim, ltrim, trim

In [0]:
# Neste ele eliminou o espaço da esquerda e trouxe o resultado
display(new_df.filter(ltrim(new_df.Cidade) == 'Belo Horizonte'))

In [0]:
# Neste ele eliminou o espaço da direita e trouxe o resultado
display(new_df.filter(rtrim(new_df.Cidade) == 'Belo Horizonte'))

In [0]:
# Neste estamos usando os 2 métodos para eliminar espaço na esquerda e direita
display(new_df.filter(rtrim(ltrim(new_df.Cidade)) == 'Belo Horizonte'))

In [0]:
# Ja elimina os 2 o melhor de usar é esse
display(new_df.filter(trim(new_df.Cidade) == 'Belo Horizonte'))

In [0]:
# Arrumando o DF com o withColumn que adiciona ou substitui uma coluna (mesmo nome substitui nome diferente adiciona)
new_df = new_df.withColumn("Cidade", trim(new_df.Cidade))
display(new_df)

In [0]:
# Filtro sem usar o trim ja foi tratado antes 
display(new_df.filter(new_df.Cidade == 'Belo Horizonte'))

### Substituindo Dados avançados 

Substituir dados em um DataFrame é uma tarefa comum durante o processo de limpeza e padronização de informações. No Databricks, é possível realizar substituições avançadas utilizando funções do PySpark, como `replace`, `regexp_replace` e outras, permitindo corrigir valores inconsistentes, padronizar formatos ou tratar erros de digitação de forma eficiente. Nesta seção, vamos explorar técnicas para substituir dados de maneira flexível, tanto em colunas específicas quanto em múltiplas colunas, utilizando expressões regulares e condições personalizadas.

In [0]:
# Simulando dados para os 27 estados do Brasil
dados = [
    ("Acre", "AC", 894470),
    ("Alagoas", "AL", 3337357),
    ("Amapá", "AP", 861773),
    ("Amazonas", "AM", 4207714),
    ("Bahia", "BA", 14930634),
    ("Ceará", "CE", 9132078),
    ("Distrito Federal", "DF", 3055149),
    ("Espírito Santo", "ES", 4018650),
    ("Goiás", "GO", 7113540),
    ("Maranhão", "MA", 7114598),
    ("Mato Grosso", "MT", 3526220),
    ("Mato Grosso do Sul", "MS", 2809394),
    ("Minas Gerais", "MG", 21168791),
    ("Pará", "PA", 8690745),
    ("Paraíba", "PB", 4039277),
    ("Paraná", "PR", 11433957),
    ("Pernambuco", "PE", 9616621),
    ("Piauí", "PI", 3273227),
    ("Rio de Janeiro", "RJ", 17366189),
    ("Rio Grande do Norte", "RN", 3534165),
    ("Rio Grande do Sul", "RS", 11422973),
    ("Rondônia", "RO", 1796460),
    ("Roraima", "RR", 631181),
    ("Santa Catarina", "SC", 7252502),
    ("São Paulo", "SP", 46289333),
    ("Sergipe", "SE", 2318822),
    ("Tocantins", "TO", 1590248),
]

colunas = ["Estado", "UF", "População"]

# Criando o DataFrame
df_estados = spark.createDataFrame(dados,colunas)
display(df_estados)

In [0]:
from pyspark.sql.functions import regexp_replace

In [0]:

# Poderia localidar a coluna tambm df_estados["Estado"] ou pelo indice
df_estados1 = df_estados.withColumn("EstadoSemAcento", regexp_replace(df_estados.Estado, "á", "a"))

display(df_estados1)

In [0]:
from pyspark.sql.functions import translate
# tem que ser na mesma ordem 
acentos     = "áàãâéèêíìóòõôúùûç"
sem_acentos = "aaaaeeeiioooouuuc"

# Aplicar a substituição usando a função translate
df_estados2 = df_estados.withColumn("EstadoSemAcento", translate(df_estados.Estado, acentos, sem_acentos))

# Exibir o DataFrame
display(df_estados2)

In [0]:
# Usando translatee lower pra nao ter problema de forma alguma tudo sem acento e minusculo
from pyspark.sql.functions import translate,lower
acentos     = "áàãâéèêíìóòõôúùûç"
sem_acentos = "aaaaeeeiioooouuuc"

df_estados3 = df_estados.withColumn("EstadoSemAcento", lower(translate(df_estados.Estado, acentos, sem_acentos)))
display(df_estados3)

In [0]:
display(df_estados3.filter(df_estados3.EstadoSemAcento == 'Maranhao' ))

In [0]:
# Usando todas as funções possiveis: sem acento, tudo minusculo e sem espaços escessivos
from pyspark.sql.functions import translate,lower,trim
acentos     = "áàãâéèêíìóòõôúùûç"
sem_acentos = "aaaaeeeiioooouuuc"

df_estados4 = df_estados.withColumn("EstadoSemAcento", trim(lower(translate(df_estados.Estado, acentos, sem_acentos))) )
display(df_estados4)

In [0]:
display(df_estados4.filter(df_estados4.EstadoSemAcento == 'sao paulo'))

In [0]:
# Subtituindo coluna e nao adicionando uma nova 
from pyspark.sql.functions import translate,lower,trim
acentos     = "áàãâéèêíìóòõôúùûç"
sem_acentos = "aaaaeeeiioooouuuc"

df_estados = df_estados.withColumn("Estado", trim(lower(translate(df_estados.Estado, acentos, sem_acentos))))
display(df_estados)

###Função Like

A função `like` é utilizada em consultas SQL e em métodos do PySpark para filtrar linhas de um DataFrame com base em padrões de texto. Ela permite buscar valores que correspondam parcialmente a um critério, utilizando curingas como `%` (qualquer sequência de caracteres) e `_` (um único caractere). Essa função é especialmente útil para localizar registros que contenham, comecem ou terminem com determinados trechos de texto, facilitando buscas flexíveis e análises exploratórias em grandes volumes de dados.

* **%Texto%** = traz todo o resto pra frente e pra tras do texto
* **Texto%** = inicia com o texto e nao importa o que tem pra frente
* **%Texto** = finaliza com o texto nao importa o que tem para tras

In [0]:
# Passar filtro por trecho especifico % a grosso modo significa todo o resto, voce vai entender na prática
resultado = df.filter(df.Descricao_do_Tipo.like("%AVE"))
display(resultado)

### Filtro de Valores

Filtrar valores em um DataFrame é uma etapa essencial para refinar a análise de dados, permitindo selecionar apenas os registros que atendem a critérios específicos. No Databricks, é possível aplicar filtros de diversas formas, seja utilizando métodos do PySpark, comandos SQL ou até mesmo recursos interativos na interface. Nesta seção, vamos apresentar as principais técnicas para filtrar valores de maneira eficiente, facilitando a obtenção de insights relevantes a partir dos dados.

In [0]:
orders_df = spark.read. \
    format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load("/Volumes/workspace/default/tutorial/bike-store/orders.csv")

display(orders_df)

In [0]:
display(orders_df.filter(orders_df.order_status == 1))

In [0]:
# Salvando em um df simulando que 1 seja pedidos pendentes 
PedidosPendentes = orders_df.filter(orders_df.order_status == 1) 
display(PedidosPendentes)

In [0]:
# Pedidos de uma loja especifica 
Loja2 = orders_df.filter(orders_df.store_id == 2) 
display(Loja2)

In [0]:
# Pedidos diferentes de uma loja espefica
fora3 = orders_df.filter(orders_df.store_id != 3) 
display(fora3)

In [0]:
# Filtro composto exemplo pedidos da Loja 3 com status =1
StatusLoja3 = orders_df.filter((orders_df.store_id == 3) & (orders_df.order_status == 1))  
display(StatusLoja3)


In [0]:
order_items = spark.read. \
    format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("/Volumes/workspace/default/tutorial/bike-store/order_items.csv")

display(order_items)

In [0]:
# Exemplos de faixa de preços
Barato = order_items.filter(order_items.list_price < 300 ) 
display(Barato)

In [0]:
Caro = order_items.filter(order_items.list_price > 2000 ) 
display(Caro)

In [0]:
medio = order_items.filter((order_items.list_price >= 700) & (order_items.list_price <= 1200))
display(medio)

In [0]:
from pyspark.sql.functions import col
medio = order_items.filter((col("list_price") >= 700) & (col("list_price") <= 1200))
display(medio)


In [0]:
ValorMinimo= 50
ValorMaximo= 100

df_resultado = order_items.filter((order_items.list_price >= ValorMinimo) & (order_items.list_price <= ValorMaximo))
display(df_resultado)